In [194]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import StandardScaler
from bed_reader import open_bed, sample_file

class Data:
    def __init__(self):
        self.gen = None
        self.index = 0

In [195]:
def simulator(allgen, sigma):
    '''
    Simulate phenotype y from X and sigmas
    '''
    X = allgen[0].gen
    N = X.shape[0]
    M = X.shape[1]
    y = np.zeros((N,1))
    sigma_epsilon=1 - sigma # environmental effect sizes
    betas = np.random.randn(M,1)*np.sqrt(sigma) # additive SNP effect sizes
    y += X@betas/np.sqrt(M) # weight 
    #print(f'sigma_epislon={sigma_epsilon}') 
    y += np.random.randn(N,1)*np.sqrt(sigma_epsilon) # add the bias
    return y, betas

def simulator_multi_bin(allgen, sigma_list):
    '''
    Simulate phenotype y from X and sigma_list
    '''
    N_indv = allgen[0].gen.shape[0]
    h = 1 - sum(sigma_list) # residual covariance

    sigma_epsilon = np.random.multivariate_normal([0] * N_indv, np.diag(np.full(N_indv, h)))

    sigma_epsilon = np.array(sigma_epsilon).reshape((len(sigma_epsilon), 1))

    y = np.zeros((N_indv,1))

    betas = []

    for i, data in enumerate(allgen):
        X = data.gen
        M = X.shape[1]
        sigma = sigma_list[i]
        beta = np.random.multivariate_normal([0] * M, np.diag(np.full(M, sigma / M)))
        beta = np.array(beta).reshape((len(beta), 1))
        betas.append(beta)
        y += X@beta
    
    y +=  sigma_epsilon 

    return y, betas

def solve_linear_equation(X, y):
    '''
    Solve least square
    '''
    sigma = np.linalg.lstsq(X, y, rcond=None)[0]
    return sigma


def solve_linear_qr(X, y):
    '''
    Solve least square using QR decomposition
    '''
    Q, R = scipy.linalg.qr(X)
    sigma = scipy.linalg.solve_triangular(R, np.dot(Q.T, y))
    return sigma

def RHE_multi_bin(allgen,y,num_random_vect=10,seed=1,verbose=False):
    '''
    RHE estimation for multi bins
    '''

    K = len(allgen)

    T = np.zeros((K+1, K+1))

    q = np.zeros((K+1, 1))

    for i, data_1 in enumerate(allgen):
        for j, data_2 in enumerate(allgen):
            gen_1 = data_1.gen
            gen_2 = data_2.gen
            N1 = gen_1.shape[0]
            N2 = gen_2.shape[0]
            assert N1 == N2
            M1 = gen_1.shape[1]
            M2 = gen_1.shape[1]
            Xi_1 = gen_1.copy()/np.sqrt(M1)
            Xi_2 = gen_2.copy()/np.sqrt(M2)
            for _ in range(num_random_vect):
                # Generate random vector to estimate trace
                rand_vector = np.random.randn(N1,1)
                T[i,j] += rand_vector.T@Xi_1@Xi_1.T@Xi_2@Xi_2.T@rand_vector/num_random_vect
    
    for i, data in enumerate(allgen):
        X = data.gen
        M = X.shape[1]
        Xi = X.copy()/np.sqrt(M)
        T[i, K] = np.trace(Xi@Xi.T)
        T[K, i] = np.trace(Xi@Xi.T)
        q[i] = y.T@Xi@Xi.T@y

    T[K, K] = allgen[0].gen.shape[0]

    q[K] = y.T@y

    print("T and q")

    print(T)

    print(q)


    if verbose:
        print(T)
    sigma_est = solve_linear_qr(T,q)
    return sigma_est

In [196]:
def simulate_geno_from_random(p_j):
    rval = np.random.random()
    dist_pj = [(1-p_j)*(1-p_j), 2*p_j*(1-p_j), p_j*p_j]
    
    if rval < dist_pj[0]:
        return 0
    elif rval >= dist_pj[0] and rval < (dist_pj[0] + dist_pj[1]):
        return 1
    else:
        return 2


def impute_geno(X):
    N = X.shape[0]
    M = X.shape[1]
    X_imp = X.copy()

    for m in range(M):
        
        observed_sum = 0
        observed_ct = 0
        for n in range(N):
            if not np.isnan(X[n, m]):
                observed_ct += 1
                observed_sum += X[n, m]
        
        observed_sum = (observed_sum  / observed_ct)* 0.5

        for j in range(N):
            if np.isnan(X[j,m]):
                X_imp[j, m] = simulate_geno_from_random(observed_sum)
                
    # standardize
    X_imp = (X_imp-np.mean(X_imp, axis=0))/np.std(X_imp, axis=0)

    return X_imp

In [197]:
geno_path="/Users/nijiayi/RHE_project/data/test2/actual_geno_1.bed"
bed = open_bed(geno_path)
X = bed.read()

In [198]:
X_imp = impute_geno(X)
Nindv = X_imp.shape[0]
Nsnp = X_imp.shape[1]

In [199]:
# Simulate annotation file
Nbin = 8

import random

def create_annot_file(Nsnp, Nbin, filename):
    with open(filename, 'w') as f:
        for _ in range(Nsnp):
            row = [0] * Nbin
            random_col = random.randint(0, Nbin - 1)
            row[random_col] = 1
            f.write(' '.join(str(val) for val in row) + '\n')

filename = '/Users/nijiayi/RHE_project/data/test2/annot.txt'
create_annot_file(Nsnp, Nbin, filename)

In [200]:
from RHE.util.file_processing import read_annot

filename = '/Users/nijiayi/RHE_project/data/test2/annot.txt'
_, annot_matrix = read_annot(filename, 1)

print(annot_matrix)

Number of SNPs per block: 996
137 SNPs in 0 -th bin
99 SNPs in 1 -th bin
115 SNPs in 2 -th bin
146 SNPs in 3 -th bin
118 SNPs in 4 -th bin
118 SNPs in 5 -th bin
135 SNPs in 6 -th bin
128 SNPs in 7 -th bin
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]


In [201]:
def bin_to_snp(annot_matrix_chunk): 
    bin_to_snp_indices = []

    for bin_index in range(annot_matrix_chunk.shape[1]):
        snp_indices = np.nonzero(annot_matrix_chunk[:, bin_index])[0]
        bin_to_snp_indices.append(snp_indices.tolist())

    return bin_to_snp_indices

bin_to_snp_map = bin_to_snp(annot_matrix)

def create_allgen(bin_to_snp_map, X):
    allgen = [Data() for _ in range(len(bin_to_snp_map))]

    for i, data in enumerate(allgen):
        data.index = len(bin_to_snp_map[i])
        data.gen = X[:, bin_to_snp_map[i]]

    return allgen

In [202]:
sigma_list = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

bin_to_snp_map = bin_to_snp(annot_matrix)
allgen = create_allgen(bin_to_snp_map, X_imp)

y, beta_list = simulator_multi_bin(allgen, sigma_list)

print(y)

print(f'Actual sigmas are: {sigma_list}')

sigma_est=RHE_multi_bin(allgen, y, num_random_vect=10, seed=42) # run py-RHE
print('RHE estimated sigmas are: {}'.format(", ".join(str(s) for s in sigma_est)))

[[-1.49803855]
 [-2.74333733]
 [-0.62589444]
 ...
 [ 1.73464368]
 [-1.0634817 ]
 [ 0.14036062]]
Actual sigmas are: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
T and q
[[ 9296.89478162   942.39161146  1200.08402358  1648.72563056
    982.64363821  1212.65025127  1395.01615746  1122.84234357
   1022.00036621]
 [ 1805.30366287 11629.12462772  1265.03406436  1970.1796114
   1741.7023196   1582.50358504  1554.15445872  1438.90377267
   1022.00085449]
 [ 1521.81556499   912.54847492  9342.57937021  1555.00066862
   1628.47704911  1407.9676057   1153.13312323  1375.11773752
   1021.99993896]
 [ 1290.74632763   844.61771047   684.14114051  8986.6357862
   1068.31275283  1042.99229441  1142.69506756  1079.32921336
   1022.00018311]
 [ 1581.02814168  1154.77127419  1194.68073757  1575.82680505
  10355.33983555  1246.43614696  1357.87010705  1389.54131524
   1022.00097656]
 [ 1593.91085105  1234.35520691  1318.86300295  1547.28580532
   1172.72057704 10711.0102295   1236.24960382  1198.43071658
   1